In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

Let:
- $M$ be the total number of features in the feature matrix $X$. These features are indexed from 1 to $M$.
- $K$ be the total number of outcomes.
- $M_{\text {shared }}$ be the number of features shared among all $K$ outcomes.
- $M_{\text {specific, } k}$ be the number of features that specifically influence outcome $k$ (for $k=$ $1, \ldots, K)$, in addition to the shared features.

- 1. Shared Feature Set ( $I_{\text {shared }}$ ):

The first $M_{\text {shared }}$ features of $X$ are designated as shared features. The set of indices for these shared features is:

$$
I_{\text {shared }}=\left\{f \mid 1 \leq f \leq M_{\text {shared }}\right\}
$$


These features influence all $K$ outcomes.  

- 2. Specific Feature Sets ( $I_{\text {specific }, k}$ ):

For each outcome $k \in\{1, \ldots, K\}$, a unique block of $M_{\text {specific, } k}$ features is allocated to specifically influence that outcome. These blocks are contiguous and follow the shared features and the specific features of any preceding outcomes.

Let $S_0=M_{\text {shared }}$.
For $k \geq 1$, the starting index for the specific features of outcome $k$ is:

$$
i d x_{s t a r t, k}=S_{k-1}+1
$$

where $S_{k-1}=M_{\text {shared }}+\sum_{l=1}^{k-1} M_{\text {specific, } l}$ (with the convention that $\sum_{l=1}^0 M_{\text {specific, } l}=0$ ).

The set of indices for the specific features for outcome $k$ is then:

$$
I_{\text {specific }, k}=\left\{f \mid i d x_{\text {start }, k} \leq f<i d x_{\text {start }, k}+M_{\text {specific }, k}\right\}
$$


The binary outcome $Y_{j, k}$ for the $j$-th sample and the $k$-th outcome is generated as follows:

$$
Y_{j, k}=\mathbb{I}\left(\left(\sum_{f \in I_{\text {rclevant, } k}} X_{j, f} \cdot w_{f, k}\right)+\epsilon_j>\theta_k\right)
$$


Where:
- $\mathbb{I}(\cdot)$ : This is the indicator function (it returns 1 if the condition inside is true, and 0 otherwise).
- $X_{j, f}$ : This is the value of the $f$-th feature for the $j$-th sample.
- $w_{f, k}$ : This is the random weight assigned to feature $f$ for its contribution to outcome $k$.
- $I_{\text {relevant }, k}$ : This is the set of indices for the features that are relevant to (influence) outcome $k$
- $\epsilon_j$ : This represents random noise added to the $j$-th sample.
- $\theta_k$ : This is the threshold calculated based on the desired proportion $P_k$ for outcome $k$ (i.e., the proportion of samples where $Y_{j, k}=1$ ).

In [2]:
def generate_simulation_data(n_samples,
                             n_total_features,
                             n_outcomes,
                             n_shared_features,
                             n_specific_features_list,
                             outcome_proportions):
    """
    生成用于多任务学习模拟的特征矩阵 X 和结果矩阵 Y。

    参数:
    n_samples (int): 生成的样本数量（行数）。
    n_total_features (int): 特征矩阵 X 的总列数。
    n_outcomes (int): 要生成的结果数量。
    n_shared_features (int): 所有结果共享的特征数量。
    n_specific_features_list (list of int): 每个结果额外受其影响的特定特征数量列表。
                                       列表长度应与 n_outcomes 相同。
    outcome_proportions (list of float): 每个结果中 outcome 为 1 的比例列表。
                                     列表长度应与 n_outcomes 相同。

    返回:
    pandas.DataFrame: 特征矩阵 X。
    pandas.DataFrame: 结果矩阵 Y。
    """

    if len(n_specific_features_list) != n_outcomes:
        raise ValueError("n_specific_features_list 的长度必须等于 n_outcomes。")
    if len(outcome_proportions) != n_outcomes:
        raise ValueError("outcome_proportions 的长度必须等于 n_outcomes。")
    if n_shared_features + sum(n_specific_features_list) > n_total_features:
        raise ValueError("共享特征和所有特定特征的总和不能超过总特征数。")

    # 1. 生成特征矩阵 X
    X = np.random.randn(n_samples, n_total_features)
    X_df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(n_total_features)])

    # 2. 生成结果矩阵 Y
    Y_data = np.zeros((n_samples, n_outcomes))

    # 定义特征索引
    shared_feature_indices = list(range(n_shared_features))
    current_specific_start_idx = n_shared_features

    for i in range(n_outcomes):
        # 确定影响当前结果的特征列
        specific_feature_indices = list(range(current_specific_start_idx,
                                            current_specific_start_idx + n_specific_features_list[i]))
        relevant_feature_indices = shared_feature_indices + specific_feature_indices

        # 为这些特征生成随机权重 (也可以根据需要设置固定的权重)
        # 权重的数量应该等于 relevant_feature_indices 的长度
        weights = np.random.randn(len(relevant_feature_indices))

        # 计算线性组合
        linear_combination = X[:, relevant_feature_indices] @ weights

        # 添加一些随机噪声，使得结果不完全由特征决定
        noise = np.random.randn(n_samples) * 1.0 # 噪声的标准差可以调整
        signal_with_noise = linear_combination + noise

        # 根据指定的比例确定阈值，以生成二元结果
        # 这里我们使用百分位数作为阈值
        # outcome_proportions[i] 意味着 (1 - outcome_proportions[i]) 的百分位数是阈值
        # 例如，如果比例是 3%，那么高于第 97 个百分位的值将被设为 1
        threshold = np.percentile(signal_with_noise, 100 * (1 - outcome_proportions[i]))
        
        Y_data[:, i] = (signal_with_noise > threshold).astype(int)

        # 更新下一个特定特征的起始索引
        current_specific_start_idx += n_specific_features_list[i]

    Y_df = pd.DataFrame(Y_data, columns=[f'outcome_{j+1}' for j in range(n_outcomes)], dtype=int)

    return X_df, Y_df

Let:
- $X_j$ be the feature vector for the $j$-th sample.
- $X_{j, I_{\text {shared }}}$ be the sub-vector of $X_j$ containing only the shared features.
- $X_{j, I_{\text {specific, } k}}$ be the sub-vector of $X_j$ containing only the specific features for outcome $k$.
- $w_{\text {shared }}$ be a vector of ones with dimension $M_{\text {shared }}$ (since weights_shared = np.ones(len(shared_feature_indices))).
- $w_{\text {specific, } k}$ be a vector of ones with dimension $M_{\text {specific, } k}$ (since weights_specific = np.ones(len(specific_feature_indices))).

1. Shared Component ( $C_{\text {shared }, j}$ ):



Let $Z_{\text {shared, } j}=X_{j, I_{\text {shared }}} \cdot w_{\text {shared }}=\sum_{f \in I_{\text {shared }}} X_{j, f}$ (since $w_{\text {shared }}$ contains ones).  
Then, the shared component for sample $j$ can be written as:

$$
C_{\text {shared }, j}=\tanh \left(0.5 \cdot Z_{\text {shared }, j}\right)+\cos \left(0.5 \cdot Z_{\text {shared }, j}\right)+\left(0.3 \cdot Z_{\text {shared }, j}\right)^2
$$

2. Specific Component $\left(C_{\text {specific }, j, k}\right)$ :


Let $Z_{\text {specific }, j, k}=X_{j, I_{\text {specific }, k}} \cdot w_{\text {specific }, k}=\sum_{f \in I_{\text {specific, } k}} X_{j, f}$ (since $w_{\text {specific }, k}$ contains ones).  

Then, the specific component for sample $j$ and outcome $k$ is:

$$
C_{\text {specific }, j, k}=Z_{\text {specific }, j, k}
$$

3. Combined Pre-Noise Signal ( $L_{j, k}$ ):

The linear_combination in the code depends on the outcome index i (which corresponds to $k-1$ if $k$ is 1 -indexed for outcomes). Let $\alpha_k$ and $\beta_k$ be the weighting factors for the shared and specific components for outcome $k$.  

- For outcome $k=1$ (i=0): $\alpha_1=1.5, \beta_1=0.5$
- For outcome $k=2$ ( $\mathrm{i}=1$ ): $\alpha_2=1.0, \beta_2=1.0$
- For outcome $k=3$ ( $\mathrm{i}=2$ ): $\alpha_3=0.5, \beta_3=1.5$

So, the combined signal before noise for sample $j$ and outcome $k$ is:

$$
L_{j, k}=\alpha_k \cdot C_{\text {shared }, j}+\beta_k \cdot C_{\text {specific }, j, k}
$$


$$
L_{j, k}=\alpha_k\left(\tanh \left(0.5 Z_{\text {shared }, j}\right)+\cos \left(0.5 Z_{\text {shared }, j}\right)+\left(0.3 Z_{\text {shared }, j}\right)^2\right)+\beta_k Z_{\text {specific }, j, k}
$$

4. Binary Outcome Generation ( $Y_{j, k}$ ):

The binary outcome $Y_{j, k}$ for the $j$-th sample and the $k$-th outcome is generated by adding noise and then thresholding:

$$
\begin{gathered}
S_{j, k}=L_{j, k}+\epsilon_j \\
Y_{j, k}=\mathbb{I}\left(S_{j, k}>\theta_k\right)
\end{gathered}
$$

Where:
- $\mathbb{I}(\cdot)$ : This is the indicator function (it returns 1 if the condition inside is true, and 0 otherwise).
- $\epsilon_j$ : This represents random noise added to the $j$-th sample, $\epsilon_j \sim \mathcal{N}\left(0, \sigma_{\text {noise }}^2\right)$ (in your code, $\sigma_{\text {noise }}=1.5$ ).
- $\theta_k$ : This is the threshold calculated based on the desired proportion $P_k$ for outcome $k$ (i.e., the proportion of samples where $Y_{j, k}=1$ ). It's the $100 \times\left(1-P_k\right)$-th percentile of the distribution of $S_{j, k}$ over all samples for outcome $k$.


In [3]:
def generate_simulation_data(n_samples,
                             n_total_features,
                             n_outcomes,
                             n_shared_features,
                             n_specific_features_list,
                             outcome_proportions):
    """
    生成用于多任务学习模拟的特征矩阵 X 和结果矩阵 Y。

    参数:
    n_samples (int): 生成的样本数量（行数）。
    n_total_features (int): 特征矩阵 X 的总列数。
    n_outcomes (int): 要生成的结果数量。
    n_shared_features (int): 所有结果共享的特征数量。
    n_specific_features_list (list of int): 每个结果额外受其影响的特定特征数量列表。
                                       列表长度应与 n_outcomes 相同。
    outcome_proportions (list of float): 每个结果中 outcome 为 1 的比例列表。
                                     列表长度应与 n_outcomes 相同。

    返回:
    pandas.DataFrame: 特征矩阵 X。
    pandas.DataFrame: 结果矩阵 Y。
    """

    if len(n_specific_features_list) != n_outcomes:
        raise ValueError("n_specific_features_list 的长度必须等于 n_outcomes。")
    if len(outcome_proportions) != n_outcomes:
        raise ValueError("outcome_proportions 的长度必须等于 n_outcomes。")
    if n_shared_features + sum(n_specific_features_list) > n_total_features:
        raise ValueError("共享特征和所有特定特征的总和不能超过总特征数。")

    # 1. 生成特征矩阵 X
    X = np.random.randn(n_samples, n_total_features)
    X_df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(n_total_features)])

    # 2. 生成结果矩阵 Y
    Y_data = np.zeros((n_samples, n_outcomes))

    # 定义特征索引
    shared_feature_indices = list(range(n_shared_features))
    current_specific_start_idx = n_shared_features

    for i in range(n_outcomes):
        specific_feature_indices = list(range(current_specific_start_idx,
                                            current_specific_start_idx + n_specific_features_list[i]))
        relevant_feature_indices = shared_feature_indices + specific_feature_indices
        
        # weights_shared = np.random.randn(len(shared_feature_indices))
        # weights_specific = np.random.randn(len(specific_feature_indices))

        weights_shared = np.ones(len(shared_feature_indices))
        weights_specific = np.ones(len(specific_feature_indices))

        # Introduce some non-linearity for shared features
        # This non-linear component is common to all tasks, but its impact is weighted
        shared_component_nonlinear = np.tanh(X[:, shared_feature_indices] @ (weights_shared * 0.5)) + \
                                     np.cos(X[:, shared_feature_indices] @ (weights_shared * 0.5)) + \
                                     np.square(X[:, shared_feature_indices] @ (weights_shared * 0.3))     

        # Linear component from specific features
        specific_component_linear = X[:, specific_feature_indices] @ weights_specific

        # Combine components - maybe shared part is more dominant or interacts
        # Let's say for outcome 1, shared is more important, for outcome 3, specific is.
        if i == 0: # Outcome 1
            linear_combination = shared_component_nonlinear * 1.5 + specific_component_linear * 0.5
        elif i == 1: # Outcome 2
            linear_combination = shared_component_nonlinear * 1.0 + specific_component_linear * 1.0
        else: # Outcome 3
            linear_combination = shared_component_nonlinear * 0.5 + specific_component_linear * 1.5

        noise = np.random.randn(n_samples) * 1.5 # Increased noise
        signal_with_noise = linear_combination + noise
        
        threshold = np.percentile(signal_with_noise, 100 * (1 - outcome_proportions[i]))
        Y_data[:, i] = (signal_with_noise > threshold).astype(int)
        
        current_specific_start_idx += n_specific_features_list[i]

    Y_df = pd.DataFrame(Y_data, columns=[f'outcome_{j+1}' for j in range(n_outcomes)], dtype=int)

    return X_df, Y_df

In [4]:
# --- 参数设置 ---
n_rows = 500  # 样本数量
n_total_cols_X = 50 # X的总列数
num_outcomes = 3   # 结果的数量

# 特征分配
shared_cols = 20  # 共享特征的数量
# 每个outcome额外单独受影响的特征数量
# outcome 1 单独受 10 列影响
# outcome 2 单独受 10 列影响
# outcome 3 单独受 10 列影响
# 20 (共享) + 10 (O1) + 10 (O2) + 10 (O3) = 50 (总特征数)
specific_cols_per_outcome = [10, 10, 10]

# 结果为1的比例
proportions_of_ones = [0.03, 0.05, 0.12] # outcome1=3%, outcome2=5%, outcome3=10%

# --- 生成数据 ---
X_sim, Y_sim = generate_simulation_data(
    n_samples=n_rows,
    n_total_features=n_total_cols_X,
    n_outcomes=num_outcomes,
    n_shared_features=shared_cols,
    n_specific_features_list=specific_cols_per_outcome,
    outcome_proportions=proportions_of_ones
)

# --- 打印一些信息和数据头部 ---
print("特征矩阵 X (前5行):")
print(X_sim.head())
print("\n结果矩阵 Y (前5行):")
print(Y_sim.head())

print("\nX的形状:", X_sim.shape)
print("Y的形状:", Y_sim.shape)

print("\n每个Outcome中1的实际比例:")
for i in range(num_outcomes):
    actual_proportion = Y_sim[f'outcome_{i+1}'].mean()
    print(f"Outcome {i+1}: {actual_proportion:.4f} (期望: {proportions_of_ones[i]})")

print("\n共享特征列名:", X_sim.columns[:shared_cols].tolist())
current_idx = shared_cols
for i in range(num_outcomes):
    print(f"Outcome {i+1} 的特定特征列名:", X_sim.columns[current_idx : current_idx + specific_cols_per_outcome[i]].tolist())
    current_idx += specific_cols_per_outcome[i]

特征矩阵 X (前5行):
   feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0   1.367207  -0.212154   0.945395   0.155082   0.897312   1.558312   
1  -1.350954   0.265787   1.766339   0.256847   0.283624  -1.040460   
2   0.585420   0.298662  -0.013306  -0.107048   0.427913   1.463934   
3   0.822995  -1.370238  -0.101783  -1.538012  -0.592612  -0.436027   
4  -2.147712   0.039841  -0.464098   0.099351  -0.933959   0.187428   

   feature_7  feature_8  feature_9  feature_10  ...  feature_41  feature_42  \
0   1.666489   0.505759   0.188817    0.836058  ...   -1.184929   -0.181021   
1  -0.957071   1.314946   1.121816   -0.665375  ...    0.525489   -0.281244   
2   0.140767   0.551922  -1.207128    0.240290  ...   -1.024939   -0.536162   
3  -0.872720  -0.108093   0.354522    0.040711  ...   -0.142007    0.673951   
4  -0.197051  -0.992226   0.204837    0.482590  ...    1.613960    0.721262   

   feature_43  feature_44  feature_45  feature_46  feature_47  feature_48  \
0   -0.

In [5]:
def calculate_accuracy(y_true, y_pred_binary):
    """计算准确率"""
    return accuracy_score(y_true, y_pred_binary)

def calculate_auc(y_true, y_pred_proba):
    """计算AUC"""
    # 确保 y_pred_proba 是正类的概率
    if y_pred_proba.ndim == 2 and y_pred_proba.shape[1] == 2:
        y_pred_proba = y_pred_proba[:, 1]
    return roc_auc_score(y_true, y_pred_proba)

In [6]:
# --- 1. 划分训练集和测试集 ---
# 我们将对整个X和Y进行一次划分，然后针对每个outcome进行训练
# stratify=Y_sim 可以在多标签情况下帮助保持类别比例，但对于非常稀疏的多标签组合可能不完美
# 如果Y_sim中有很多全零或全一的行，或者标签组合非常多，stratify可能效果不佳或报错
# 在这种情况下，可以考虑不使用stratify，或者对每个outcome单独划分（但这样X_train/X_test会不同）
# 这里我们尝试对Y进行整体分层，如果报错，可以改成 stratify=None
try:
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_sim, Y_sim, test_size=0.2, random_state=42, stratify=None
    )
except ValueError as e:
    print(f"分层抽样失败: {e}. 将尝试不使用分层。")
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_sim, Y_sim, test_size=0.2, random_state=42
    )

In [7]:
baseline_results = {}

for i in range(num_outcomes):
    outcome_col_name = f'outcome_{i+1}'
    print(f"\n--- 训练和评估 Outcome: {outcome_col_name} ---")

    y_train_outcome = Y_train[outcome_col_name]
    y_test_outcome = Y_test[outcome_col_name]

    # 检查训练集中是否有足够的多样性
    if len(np.unique(y_train_outcome)) < 2:
        print(f"Outcome {outcome_col_name} 在训练集中只有一个类别，跳过此outcome。")
        continue

    # --- 2. 初始化和训练逻辑回归模型 ---
    # L2正则化默认开启，C是正则化强度的倒数，较小的C表示更强的正则化
    # solver='liblinear' 适用于小型数据集和二分类问题
    # class_weight='balanced' 对于类别不平衡的数据集有帮助
    model = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42, C=1.0)
    model.fit(X_train, y_train_outcome)

    # --- 3. 获取beta系数 ---
    # model.coef_ 是一个二维数组 (1, n_features)
    betas = model.coef_[0]
    intercept = model.intercept_[0]
    print(f"模型截距 (Intercept) for {outcome_col_name}: {intercept:.4f}")
    # print(f"模型系数 (Betas) for {outcome_col_name}:\n{betas}") # 如果特征很多，打印会很长

    # --- 4. 在测试集上进行预测 ---
    y_pred_proba_outcome = model.predict_proba(X_test) # 获取概率
    y_pred_binary_outcome = model.predict(X_test)      # 获取二元预测 (0或1)

    # --- 5. 计算并记录 Accuracy 和 AUC ---
    accuracy = calculate_accuracy(y_test_outcome, y_pred_binary_outcome)
    # predict_proba 返回两列，第二列是正类（1）的概率
    auc = calculate_auc(y_test_outcome, y_pred_proba_outcome[:, 1])

    baseline_results[outcome_col_name] = {
        'accuracy': accuracy,
        'auc': auc,
        'betas': betas,
        'intercept': intercept
    }

    print(f"测试集 Accuracy for {outcome_col_name}: {accuracy:.4f}")
    print(f"测试集 AUC for {outcome_col_name}: {auc:.4f}")

print("\n\n--- 单任务基线模型结果总结 ---")
for outcome, metrics in baseline_results.items():
    print(f"\nOutcome: {outcome}")
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  AUC: {metrics['auc']:.4f}")


--- 训练和评估 Outcome: outcome_1 ---
模型截距 (Intercept) for outcome_1: -4.8124
测试集 Accuracy for outcome_1: 0.9100
测试集 AUC for outcome_1: 0.6186

--- 训练和评估 Outcome: outcome_2 ---
模型截距 (Intercept) for outcome_2: -4.5774
测试集 Accuracy for outcome_2: 0.8900
测试集 AUC for outcome_2: 0.6000

--- 训练和评估 Outcome: outcome_3 ---
模型截距 (Intercept) for outcome_3: -3.3899
测试集 Accuracy for outcome_3: 0.9100
测试集 AUC for outcome_3: 0.9357


--- 单任务基线模型结果总结 ---

Outcome: outcome_1
  Accuracy: 0.9100
  AUC: 0.6186

Outcome: outcome_2
  Accuracy: 0.8900
  AUC: 0.6000

Outcome: outcome_3
  Accuracy: 0.9100
  AUC: 0.9357


In [8]:
lasso_results = {}
# C值是正则化强度的倒数。较小的值表示更强的正则化。
# 您可以尝试不同的C值来观察其对特征选择和模型性能的影响。
C_value_lasso = 0.1 # 这是一个可以调整的超参数

for i in range(num_outcomes):
    outcome_col_name = f'outcome_{i+1}'
    print(f"\n--- 训练和评估 Outcome: {outcome_col_name} 使用Lasso (C={C_value_lasso}) ---")

    y_train_outcome = Y_train[outcome_col_name]
    y_test_outcome = Y_test[outcome_col_name]

    if len(np.unique(y_train_outcome)) < 2:
        print(f"Outcome {outcome_col_name} 在训练集中只有一个类别，跳过此outcome。")
        continue

    # --- 初始化和训练带L1正则化的逻辑回归模型 ---
    # penalty='l1' 指定使用L1正则化 (Lasso)
    # solver='liblinear' 或 'saga' 支持L1正则化。'liblinear' 通常对小型数据集表现良好。
    # 对于 'saga'，可能需要增加 max_iter。
    model_lasso = LogisticRegression(
        penalty='l1',
        solver='liblinear', # 或者 'saga'
        class_weight='balanced',
        random_state=42,
        C=C_value_lasso,
        # max_iter=1000 # 如果使用 'saga' 且不收敛，可以增加此参数
    )
    model_lasso.fit(X_train, y_train_outcome)

    # --- 获取beta系数 ---
    betas_lasso = model_lasso.coef_[0]
    intercept_lasso = model_lasso.intercept_[0]
    print(f"Lasso模型截距 for {outcome_col_name}: {intercept_lasso:.4f}")
    # print(f"Lasso模型系数 for {outcome_col_name}:\n{betas_lasso}")

    num_zero_betas = np.sum(np.abs(betas_lasso) < 1e-6) # 计算近似为零的系数数量
    print(f"Lasso模型中接近零的Beta系数数量 for {outcome_col_name}: {num_zero_betas} / {len(betas_lasso)}")


    # --- 在测试集上进行预测 ---
    y_pred_proba_lasso = model_lasso.predict_proba(X_test)[:, 1] # 正类的概率
    y_pred_binary_lasso = model_lasso.predict(X_test)

    # --- 计算并记录 Accuracy 和 AUC ---
    accuracy_lasso = calculate_accuracy(y_test_outcome, y_pred_binary_lasso)
    auc_lasso = calculate_auc(y_test_outcome, y_pred_proba_lasso)

    lasso_results[outcome_col_name] = {
        'accuracy': accuracy_lasso,
        'auc': auc_lasso,
        'betas': betas_lasso,
        'intercept': intercept_lasso,
        'num_zero_betas': num_zero_betas
    }

    print(f"测试集 Lasso Accuracy for {outcome_col_name}: {accuracy_lasso:.4f}")
    print(f"测试集 Lasso AUC for {outcome_col_name}: {auc_lasso:.4f}")

    # 可选：绘制ROC曲线 (与之前代码类似)
    # fpr_lasso, tpr_lasso, _ = roc_curve(y_test_outcome, y_pred_proba_lasso)
    # plt.figure()
    # plt.plot(fpr_lasso, tpr_lasso, label=f'{outcome_col_name} Lasso (area = {auc_lasso:.2f})')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0.0, 1.0])
    # plt.ylim([0.0, 1.05])
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title(f'Lasso ROC Curve for {outcome_col_name}')
    # plt.legend(loc="lower right")
    # plt.show()

print("\n\n--- Lasso (L1正则化逻辑回归) 单任务模型结果总结 ---")
for outcome, metrics in lasso_results.items():
    print(f"\nOutcome: {outcome}")
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  AUC: {metrics['auc']:.4f}")
    print(f"  接近零的Beta系数数量: {metrics['num_zero_betas']} / {n_total_cols_X}")
    # 打印非零系数对应的特征名
    print(f"  非零系数的特征:")
    non_zero_indices = np.where(np.abs(metrics['betas']) > 1e-6)[0]
    if len(non_zero_indices) > 0:
        for idx in non_zero_indices:
            print(f"    {X_sim.columns[idx]}: {metrics['betas'][idx]:.4f}")
    else:
        print("    所有特征系数均接近零。")


--- 训练和评估 Outcome: outcome_1 使用Lasso (C=0.1) ---
Lasso模型截距 for outcome_1: -1.2730
Lasso模型中接近零的Beta系数数量 for outcome_1: 22 / 50
测试集 Lasso Accuracy for outcome_1: 0.7800
测试集 Lasso AUC for outcome_1: 0.6151

--- 训练和评估 Outcome: outcome_2 使用Lasso (C=0.1) ---
Lasso模型截距 for outcome_2: -1.3561
Lasso模型中接近零的Beta系数数量 for outcome_2: 26 / 50
测试集 Lasso Accuracy for outcome_2: 0.8700
测试集 Lasso AUC for outcome_2: 0.6337

--- 训练和评估 Outcome: outcome_3 使用Lasso (C=0.1) ---
Lasso模型截距 for outcome_3: -0.8318
Lasso模型中接近零的Beta系数数量 for outcome_3: 27 / 50
测试集 Lasso Accuracy for outcome_3: 0.9100
测试集 Lasso AUC for outcome_3: 0.9851


--- Lasso (L1正则化逻辑回归) 单任务模型结果总结 ---

Outcome: outcome_1
  Accuracy: 0.7800
  AUC: 0.6151
  接近零的Beta系数数量: 22 / 50
  非零系数的特征:
    feature_1: 0.5435
    feature_3: 0.1188
    feature_4: 0.0030
    feature_6: 0.3325
    feature_7: -0.0875
    feature_8: 0.3353
    feature_9: 0.2576
    feature_11: 0.3174
    feature_12: 0.1980
    feature_17: 0.0539
    feature_20: 0.5158
    feature_24:

In [9]:
# --- 新增：DNN 模型创建函数 ---
def create_dnn_model(input_dim, hidden_units=[64, 32], dropout_rate=0.3):
    """
    创建一个简单的DNN模型用于二分类。

    参数:
    input_dim (int): 输入特征的数量。
    hidden_units (list of int): 每个隐藏层中的神经元数量。
    dropout_rate (float): Dropout比率。

    返回:
    tensorflow.keras.models.Sequential: 编译好的Keras模型。
    """
    model = Sequential()
    model.add(Dense(hidden_units[0], input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    if len(hidden_units) > 1:
        for units in hidden_units[1:]:
            model.add(Dense(units, activation='relu'))
            model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid')) # 二分类输出层

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

dnn_results = {}
# DNN训练参数
epochs = 50
batch_size = 32
# 定义提前停止回调
early_stopping = EarlyStopping(monitor='val_auc', mode='max', patience=10, restore_best_weights=True, verbose=1)


for i in range(num_outcomes):
    outcome_col_name = f'outcome_{i+1}'
    print(f"\n--- 训练和评估 Outcome: {outcome_col_name} 使用DNN ---")

    y_train_outcome = Y_train[outcome_col_name].values # Keras 通常期望 numpy array
    y_test_outcome = Y_test[outcome_col_name].values

    if len(np.unique(y_train_outcome)) < 2:
        print(f"Outcome {outcome_col_name} 在训练集中只有一个类别，跳过此outcome。")
        dnn_results[outcome_col_name] = {'accuracy': np.nan, 'auc': np.nan, 'history': None, 'weights': None}
        continue

    # --- 创建和训练DNN模型 ---
    dnn_model = create_dnn_model(input_dim=X_train.shape[1])
    print(dnn_model.summary())

    # 计算类别权重以处理不平衡数据
    neg_count = np.sum(y_train_outcome == 0)
    pos_count = np.sum(y_train_outcome == 1)
    total = neg_count + pos_count

    # 权重计算公式: weight_for_0 = (1 / neg_count) * (total / 2.0)
    #              weight_for_1 = (1 / pos_count) * (total / 2.0)
    # Keras期望一个字典: {class_0_label: weight_0, class_1_label: weight_1}
    class_weights = {
        0: (1 / neg_count) * (total / 2.0) if neg_count > 0 else 1,
        1: (1 / pos_count) * (total / 2.0) if pos_count > 0 else 1
    }
    print(f"类别权重 for {outcome_col_name}: {class_weights}")


    history = dnn_model.fit(
        X_train,
        y_train_outcome,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2, # 从训练集中分出一部分作为验证集
        callbacks=[early_stopping],
        class_weight=class_weights, # 应用类别权重
        verbose=1 # 设置为0以减少训练过程中的输出，设置为1或2以查看进度
    )

    # --- 获取模型权重 (可选，DNN的权重解释性不如线性模型) ---
    # 这里可以获取第一层权重作为示例
    first_layer_weights = dnn_model.layers[0].get_weights()[0] # [0]是权重矩阵，[1]是偏置

    # --- 在测试集上进行评估 ---
    # predict返回的是(0,1)之间的概率值
    y_pred_proba_dnn = dnn_model.predict(X_test)
    # 将概率转换为二元预测 (阈值0.5)
    y_pred_binary_dnn = (y_pred_proba_dnn > 0.5).astype(int).flatten() # flatten() 以匹配y_test_outcome的形状

    # --- 计算并记录 Accuracy 和 AUC ---
    accuracy_dnn = calculate_accuracy(y_test_outcome, y_pred_binary_dnn)
    auc_dnn = calculate_auc(y_test_outcome, y_pred_proba_dnn) # y_pred_proba_dnn已经是正类的概率

    dnn_results[outcome_col_name] = {
        'accuracy': accuracy_dnn,
        'auc': auc_dnn,
        'history': history.history, # 存储训练历史，方便后续绘图
        'weights_layer0': first_layer_weights
    }

    print(f"测试集 DNN Accuracy for {outcome_col_name}: {accuracy_dnn:.4f}")
    print(f"测试集 DNN AUC for {outcome_col_name}: {auc_dnn:.4f}")

    # 可选：绘制训练过程中的损失和AUC曲线
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(history.history['loss'], label='Train Loss')
    # plt.plot(history.history['val_loss'], label='Validation Loss')
    # plt.title(f'Loss for {outcome_col_name}')
    # plt.xlabel('Epochs')
    # plt.ylabel('Loss')
    # plt.legend()

    # plt.subplot(1, 2, 2)
    # plt.plot(history.history['auc'], label='Train AUC')
    # plt.plot(history.history['val_auc'], label='Validation AUC')
    # plt.title(f'AUC for {outcome_col_name}')
    # plt.xlabel('Epochs')
    # plt.ylabel('AUC')
    # plt.legend()
    # plt.tight_layout()
    # plt.show()


print("\n\n--- DNN 单任务模型结果总结 ---")
for outcome, metrics in dnn_results.items():
    if metrics['history'] is not None: # 仅打印成功训练的outcome
        print(f"\nOutcome: {outcome}")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  AUC: {metrics['auc']:.4f}")
        print(f"  训练停止时的验证集AUC (best): {max(metrics['history'].get('val_auc', [np.nan])):.4f}")
        # print(f"  第一层权重形状: {metrics['weights_layer0'].shape if metrics['weights_layer0'] is not None else 'N/A'}")
    else:
        print(f"\nOutcome: {outcome} - 训练跳过")


--- 训练和评估 Outcome: outcome_1 使用DNN ---
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3264      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5377 (21.00 KB)
Trainable params: 5377 (21.00 KB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________

In [10]:
# --- 深度学习库导入 ---
import tensorflow as tf
from tensorflow.keras.models import Model # Functional API 用于多输出模型
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model # 用于可视化模型结构，可选

# --- 新增：MTL DNN 模型创建函数 ---
def create_mtl_dnn_model(input_dim, num_outcomes, shared_hidden_units=[64, 32], task_specific_hidden_units=[16], dropout_rate=0.3):
    """
    创建一个多任务学习的DNN模型。

    参数:
    input_dim (int): 输入特征的数量。
    num_outcomes (int): 任务（结果）的数量。
    shared_hidden_units (list of int): 共享隐藏层中的神经元数量。
    task_specific_hidden_units (list of int): 每个任务特定分支的隐藏层神经元数量（可选，如果为空则直接连接到输出）。
    dropout_rate (float): Dropout比率。

    返回:
    tensorflow.keras.models.Model: 编译好的Keras MTL模型。
    """
    # 输入层
    input_layer = Input(shape=(input_dim,), name='input_features')

    # 共享层
    shared_layer = input_layer
    for units in shared_hidden_units:
        shared_layer = Dense(units, activation='relu')(shared_layer)
        shared_layer = Dropout(dropout_rate)(shared_layer)

    # 任务特定的输出层
    output_layers = []
    for i in range(num_outcomes):
        task_layer = shared_layer # 从共享层的输出开始
        # 可以为每个任务添加额外的特定隐藏层
        for units in task_specific_hidden_units:
            task_layer = Dense(units, activation='relu', name=f'task_{i+1}_specific_dense_{units}')(task_layer)
            task_layer = Dropout(dropout_rate, name=f'task_{i+1}_specific_dropout')(task_layer)
        output_layer = Dense(1, activation='sigmoid', name=f'outcome_{i+1}')(task_layer)
        output_layers.append(output_layer)

    model = Model(inputs=input_layer, outputs=output_layers, name='mtl_dnn_model')

    # 编译模型
    # 每个输出层需要一个损失函数和对应的指标
    # 如果所有任务都是二分类，损失函数都是 'binary_crossentropy'
    losses = {f'outcome_{i+1}': 'binary_crossentropy' for i in range(num_outcomes)}
    metrics = {f'outcome_{i+1}': ['accuracy', tf.keras.metrics.AUC(name=f'auc_{i+1}')] for i in range(num_outcomes)}
    # 可以为不同的任务设置不同的损失权重
    loss_weights = {f'outcome_{i+1}': 1.0 for i in range(num_outcomes)} # 默认权重相同

    model.compile(optimizer='adam',
                  loss=losses,
                  metrics=metrics,
                  loss_weights=loss_weights)
    return model



# Keras MTL 模型期望Y是列表或字典形式，每个元素对应一个输出
Y_train_list = [Y_train[f'outcome_{i+1}'].values for i in range(num_outcomes)]
Y_test_list = [Y_test[f'outcome_{i+1}'].values for i in range(num_outcomes)]
# 或者使用字典形式，与模型编译时的loss和metrics的key对应
Y_train_dict = {f'outcome_{i+1}': Y_train[f'outcome_{i+1}'].values for i in range(num_outcomes)}
Y_test_dict = {f'outcome_{i+1}': Y_test[f'outcome_{i+1}'].values for i in range(num_outcomes)}


print("\n--- 开始使用MTL DNN模型训练与评估 ---")

mtl_dnn_results = {}
epochs_mtl = 60 # MTL可能需要更多epochs来学习共享表示和特定任务
batch_size_mtl = 32
# 定义提前停止回调，监控整体验证损失或特定任务的验证AUC
# 这里监控 val_loss (整体加权损失)
early_stopping_mtl = EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True, verbose=1)

# --- 创建和训练MTL DNN模型 ---
mtl_model = create_mtl_dnn_model(
    input_dim=X_train.shape[1],
    num_outcomes=num_outcomes,
    shared_hidden_units=[128, 64], # 可以尝试不同的共享层配置
    task_specific_hidden_units=[32], # 每个任务分支再加一个32神经元的隐藏层
    dropout_rate=0.4
)
print(mtl_model.summary())
# plot_model(mtl_model, to_file='mtl_dnn_model.png', show_shapes=True, show_layer_names=True) # 可选：保存模型结构图

history_mtl = mtl_model.fit(
    X_train,
    Y_train_dict,
    epochs=epochs_mtl,
    batch_size=batch_size_mtl,
    validation_split=0.2,
    callbacks=[early_stopping_mtl],
    class_weight=None, # 暂时移除 class_weight
    verbose=1
)

y_pred_proba_mtl_list = mtl_model.predict(X_test)

print("\n--- MTL DNN模型在测试集上的性能 ---")
for i in range(num_outcomes):
    outcome_col_name = f'outcome_{i+1}'
    y_test_outcome_single = Y_test_dict[outcome_col_name]
    y_pred_proba_single = y_pred_proba_mtl_list[i]
    y_pred_binary_single = (y_pred_proba_single > 0.5).astype(int).flatten()

    accuracy_mtl = calculate_accuracy(y_test_outcome_single, y_pred_binary_single)
    auc_mtl = calculate_auc(y_test_outcome_single, y_pred_proba_single)

    mtl_dnn_results[outcome_col_name] = {
        'accuracy': accuracy_mtl,
        'auc': auc_mtl
    }
    print(f"Outcome: {outcome_col_name}")
    print(f"  Accuracy: {accuracy_mtl:.4f}")
    print(f"  AUC: {auc_mtl:.4f}")


--- 开始使用MTL DNN模型训练与评估 ---
Model: "mtl_dnn_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 50)]                 0         []                            
 )                                                                                                
                                                                                                  
 dense_9 (Dense)             (None, 128)                  6528      ['input_features[0][0]']      
                                                                                                  
 dropout_6 (Dropout)         (None, 128)                  0         ['dense_9[0][0]']             
                                                                                                  
 dense_10 (Dense)            (None, 64)                   

In [11]:
def display_results_table(baseline_res, lasso_res, dnn_res, mtl_dnn_res, total_features):
    """
    将所有模型的结果汇总到一个表格中并打印。

    参数:
    baseline_res (dict): 逻辑回归（L2）的结果字典。
    lasso_res (dict): Lasso逻辑回归（L1）的结果字典。
    dnn_res (dict): 单任务DNN的结果字典。
    mtl_dnn_res (dict): 多任务DNN的结果字典。
    total_features (int): 特征总数，用于显示Lasso的稀疏性。
    """
    
    print("\n\n---Accuracy (AUC)---")

    summary_data = []
    outcome_names = [f'outcome_{i+1}' for i in range(num_outcomes)]

    for outcome_name in outcome_names:
        row = {'Outcome': outcome_name}

        # Logistic Regression (Baseline)
        if outcome_name in baseline_results and baseline_results[outcome_name].get('accuracy') is not None:
            acc_lr = baseline_results[outcome_name]['accuracy']
            auc_lr = baseline_results[outcome_name]['auc']
            row['Logistic Regression'] = f"{acc_lr:.4f} ({auc_lr:.4f})"
        else:
            row['Logistic Regression'] = "N/A (skipped)"

        # Lasso (L1 LogReg)
        if outcome_name in lasso_results and lasso_results[outcome_name].get('accuracy') is not None:
            acc_lasso = lasso_results[outcome_name]['accuracy']
            auc_lasso = lasso_results[outcome_name]['auc']
            row['Lasso (L1 LogReg)'] = f"{acc_lasso:.4f} ({auc_lasso:.4f})"
        else:
            row['Lasso (L1 LogReg)'] = "N/A (skipped)"

        # DNN (Single Task)
        if outcome_name in dnn_results and dnn_results[outcome_name].get('accuracy') is not None and not np.isnan(dnn_results[outcome_name]['accuracy']):
            acc_dnn_stl = dnn_results[outcome_name]['accuracy']
            auc_dnn_stl = dnn_results[outcome_name]['auc']
            row['DNN (Single Task)'] = f"{acc_dnn_stl:.4f} ({auc_dnn_stl:.4f})"
        else:
            row['DNN (Single Task)'] = "N/A (skipped)"
            if outcome_name in dnn_results and dnn_results[outcome_name].get('accuracy') is not None and np.isnan(dnn_results[outcome_name]['accuracy']):
                 row['DNN (Single Task)'] = "N/A (NaN result)"


        # MTL DNN
        if outcome_name in mtl_dnn_results and mtl_dnn_results[outcome_name].get('accuracy') is not None:
            acc_dnn_mtl = mtl_dnn_results[outcome_name]['accuracy']
            auc_dnn_mtl = mtl_dnn_results[outcome_name]['auc']
            row['MTL DNN'] = f"{acc_dnn_mtl:.4f} ({auc_dnn_mtl:.4f})"
        else:
            # MTL DNN 的结果是针对所有任务的，如果某个任务在训练时就有问题（例如类别不平衡到无法训练特定头部）
            # 或者如果在评估时没有正确填充mtl_dnn_results，这里也会显示N/A
            row['MTL DNN'] = "N/A"


        summary_data.append(row)

    summary_df = pd.DataFrame(summary_data)
    summary_df.set_index('Outcome', inplace=True)

    print(summary_df.to_string()) # 使用 to_string() 保证表格完整打印

display_results_table(baseline_results, lasso_results, dnn_results, mtl_dnn_results, n_total_cols_X)



---Accuracy (AUC)---
          Logistic Regression Lasso (L1 LogReg) DNN (Single Task)          MTL DNN
Outcome                                                                           
outcome_1     0.9100 (0.6186)   0.7800 (0.6151)   0.9500 (0.6151)  0.9700 (0.6701)
outcome_2     0.8900 (0.6000)   0.8700 (0.6337)   0.8800 (0.4547)  0.9500 (0.4379)
outcome_3     0.9100 (0.9357)   0.9100 (0.9851)   0.9500 (0.9216)  0.8500 (0.7953)
